In [1]:

from utils.path_utils import project_root

import os

import torch

import numpy as np
import pandas as pd

import tqdm


In [2]:
import argparse

parser = argparse.ArgumentParser()

home_dir = os.getcwd()
parser.add_argument('--run_description', default='run1', type=str, help='Experiment Description')
parser.add_argument('--seed', default=2023, type=int, help='seed value')

parser.add_argument('--training_mode', default='pre_train', type=str, help='pre_train, fine_tune')
parser.add_argument('--pretrain_dataset', default='SleepEEG', type=str,
                    help='Dataset of choice: SleepEEG, FD_A, HAR, ECG')
parser.add_argument('--target_dataset', default='Epilepsy', type=str,
                    help='Dataset of choice: Epilepsy, FD_B, Gesture, EMG')

parser.add_argument('--logs_save_dir', default='experiments_logs', type=str, help='saving directory')
parser.add_argument('--device', default='cuda', type=str, help='cpu or cuda')
parser.add_argument('--home_path', default=home_dir, type=str, help='Project home directory')
parser.add_argument('--subset', action='store_true', default=False, help='use the subset of datasets')
parser.add_argument('--log_epoch', default=5, type=int, help='print loss and metrix')
parser.add_argument('--draw_similar_matrix', default=10, type=int, help='draw similarity matrix')
parser.add_argument('--pretrain_lr', default=0.0001, type=float, help='pretrain learning rate')
parser.add_argument('--lr', default=0.0001, type=float, help='learning rate')
parser.add_argument('--use_pretrain_epoch_dir', default=None, type=str,
                    help='choose the pretrain checkpoint to finetune')
parser.add_argument('--pretrain_epoch', default=10, type=int, help='pretrain epochs')
parser.add_argument('--finetune_epoch', default=300, type=int, help='finetune epochs')

parser.add_argument('--masking_ratio', default=0.5, type=float, help='masking ratio')
parser.add_argument('--positive_nums', default=3, type=int, help='positive series numbers')
parser.add_argument('--lm', default=3, type=int, help='average masked lenght')

parser.add_argument('--finetune_result_file_name', default="finetune_result.json", type=str,
                    help='finetune result json name')
parser.add_argument('--temperature', type=float, default=0.2, help='temperature')

args, unknown = parser.parse_known_args()


In [3]:

def csv_to_pt():
    patient_dir = os.path.join(project_root(), 'data', 'pt_files')
    patient_files = sorted(os.listdir(patient_dir))
    
    lengths = pd.read_csv(os.path.join(project_root(), 'data', 'processed', 'lengths.txt'), header=None).values
    is_sepsis = pd.read_csv(os.path.join(project_root(), 'data', 'processed', 'is_sepsis.txt'), header=None).values
    
    all_patients = {'samples': [], 'labels': []}
    
    max_time_step = 336
    for idx, (file_name, length, sepsis) in tqdm.tqdm(enumerate(zip(patient_files, lengths, is_sepsis)), 
                                                      desc="Converting csv to .pt format: ", 
                                                      total=len(patient_files)):
        
        file = pd.read_csv(os.path.join(patient_dir, file_name))
        
        pad_width = ((0, max_time_step - len(file)), (0, 0))
        file = np.pad(file, pad_width=pad_width, mode='constant').astype(np.float32)
        
        if len(file) == max_time_step:
            all_patients['samples'].append(torch.from_numpy(file).unsqueeze(0))
            all_patients['labels'].append(torch.tensor(sepsis[0], dtype=torch.float32).unsqueeze(0))
        else:
            raise ValueError(f"Length {length} does not match length of patient {file_name} with length {len(file)}")
    
    print('samples: ', type(all_patients['samples']), 'labels: ', type(all_patients['labels']))
    
    all_patients['samples'] = torch.cat(all_patients['samples'], dim=0)
    all_patients['labels'] = torch.cat(all_patients['labels'], dim=0)
    
    return {'samples': all_patients['samples'], 'labels': all_patients['labels']}, lengths, is_sepsis

all_patients, lengths, is_sepsis = csv_to_pt()

# del all_patients, lengths, is_sepsis


Converting csv to .pt format: 100%|██████████| 20336/20336 [00:19<00:00, 1041.88it/s]


samples:  <class 'list'> labels:  <class 'list'>


In [4]:

import math

def geom_noise_mask_single(L, lm, masking_ratio):
    """
    Randomly create a boolean mask of length `L`, consisting of subsequences of average length lm, masking with 0s a `masking_ratio`
    proportion of the sequence L. The length of masking subsequences and intervals follow a geometric distribution.
    Args:
        L: length of mask and sequence to be masked
        lm: average length of masking subsequences (streaks of 0s)
        masking_ratio: proportion of L to be masked
    Returns:
        (L, ) boolean numpy array intended to mask ('drop') with 0s a sequence of length L
    """
    keep_mask = np.ones(L, dtype=bool)
    p_m = 1 / lm  # probability of each masking sequence stopping. parameter of geometric distribution.
    p_u = p_m * masking_ratio / (
            1 - masking_ratio)  # probability of each unmasked sequence stopping. parameter of geometric distribution.
    p = [p_m, p_u]

    # Start in state 0 with masking_ratio probability
    state = int(np.random.rand() > masking_ratio)  # state 0 means masking, 1 means not masking
    for i in range(L):
        keep_mask[i] = state  # here it happens that state and masking value corresponding to state are identical
        if np.random.rand() < p[state]:
            state = 1 - state

    return keep_mask


def noise_mask(X, masking_ratio=0.25, lm=3, distribution='geometric', exclude_feats=None):
    """
    Creates a random boolean mask of the same shape as X, with 0s at places where a feature should be masked.
    Args:
        X: (seq_length, feat_dim) numpy array of features corresponding to a single sample
        masking_ratio: proportion of seq_length to be masked. At each time step, will also be the proportion of
            feat_dim that will be masked on average
        lm: average length of masking subsequences (streaks of 0s). Used only when `distribution` is 'geometric'.
        distribution: whether each mask sequence element is sampled independently at random, or whether
            sampling follows a markov chain (and thus is stateful), resulting in geometric distributions of
            masked squences of a desired mean length `lm`
        exclude_feats: iterable of indices corresponding to features to be excluded from masking (i.e. to remain all 1s)
    Returns:
        boolean numpy array with the same shape as X, with 0s at places where a feature should be masked
    """
    if exclude_feats is not None:
        exclude_feats = set(exclude_feats)

    if distribution == 'geometric':  # stateful (Markov chain)
        mask = geom_noise_mask_single(X.shape[0] * X.shape[1] * X.shape[2], lm, masking_ratio)
        mask = mask.reshape(X.shape[0], X.shape[1], X.shape[2])
        
    elif distribution == 'masked_tail':
        mask = np.ones(X.shape, dtype=bool)
        for m in range(X.shape[0]):  # feature dimension

            keep_mask = np.zeros_like(mask[m, :], dtype=bool)
            n = math.ceil(keep_mask.shape[1] * (1 - masking_ratio))
            keep_mask[:, :n] = True
            mask[m, :] = keep_mask  # time dimension
            
    elif distribution == 'masked_head':
        mask = np.ones(X.shape, dtype=bool)
        for m in range(X.shape[0]):  # feature dimension

            keep_mask = np.zeros_like(mask[m, :], dtype=bool)
            n = math.ceil(keep_mask.shape[1] * masking_ratio)
            keep_mask[:, n:] = True
            mask[m, :] = keep_mask  # time dimension
    else:  # each position is independent Bernoulli with p = 1 - masking_ratio
        mask = np.random.choice(np.array([True, False]), size=X.shape, replace=True,
                                p=(1 - masking_ratio, masking_ratio))

    return torch.tensor(mask)

def data_transform_masked4cl(sample, masking_ratio, lm, positive_nums=None, distribution='geometric'):
    """Masked time series in time dimension"""

    if positive_nums is None:
        positive_nums = math.ceil(1.5 / (1 - masking_ratio))
        
    sample = sample.permute(0, 2, 1)  # (batch_size, channels, time_steps)
    
    # Creating the batch in #positive_nums sets
    sample_repeat = sample.repeat(positive_nums, 1, 1)  # (batch_size*positive_num, channels, time steps)

    mask = noise_mask(sample_repeat, masking_ratio, lm, distribution=distribution)
    x_masked = mask * sample_repeat

    return x_masked.permute(0, 2, 1), mask.permute(0, 2, 1)

# data_masked_m, mask = data_transform_masked4cl(all_patients['samples'][:32], 0.5, 3, positive_nums=1, distribution='geometric')


In [5]:
from torch.utils.data import Dataset


class Load_Dataset(Dataset):
    
    def __init__(self, dataset, TSlength_aligned, training_mode):
        
        super(Load_Dataset, self).__init__()
        self.training_mode = training_mode
        
        X_train = dataset["samples"]
        y_train = dataset["labels"]
        
        # shuffle
        data = list(zip(X_train, y_train))
        np.random.shuffle(data)
        
        X_train, y_train = zip(*data)
        X_train, y_train = torch.stack(list(X_train), dim=0), torch.stack(list(y_train), dim=0)

        if len(X_train.shape) < 3:
            X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape)) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        """Align the TS length between source and target datasets"""
        # X_train = X_train[:, :1, :int(config.TSlength_aligned)] # take the first 178 samples
        X_train = X_train[:, :, :int(TSlength_aligned)]
        
        if isinstance(X_train, np.ndarray):
            self.x_data = torch.from_numpy(X_train)
            self.y_data = torch.from_numpy(y_train).long()
        else:
            self.x_data = X_train
            self.y_data = y_train

        self.len = X_train.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    

In [6]:
import math

from torch import nn
import torch
from torch.nn import ModuleList

import torch.nn.functional as F

from models.simmtm.gtn.encoder import Encoder
from simmtm.loss import ContrastiveWeight, AggregationRebuild, AutomaticWeightedLoss


class TFC(nn.Module):
    def __init__(self, configs, args):
        
        super(TFC, self).__init__()
        self.training_mode = 'pre_train'
        
        # Projecting input into deep representations
        self.encoder_list_1 = ModuleList([Encoder(d_model=configs.d_model, d_hidden=configs.d_hidden, q=configs.q,
                                                  v=configs.v, h=configs.h, mask=configs.mask, dropout=configs.dropout,
                                                  device=configs.device) for _ in range(configs.N)])

        self.encoder_list_2 = ModuleList([Encoder(d_model=configs.d_model, d_hidden=configs.d_hidden, q=configs.q,
                                                  v=configs.v, h=configs.h, dropout=configs.dropout,
                                                  device=configs.device) for _ in range(configs.N)])

        self.embedding_channel = torch.nn.Linear(configs.d_channel, configs.d_model)
        self.embedding_input = torch.nn.Linear(configs.d_input, configs.d_model)

        self.gate = torch.nn.Linear(configs.d_model * configs.d_input + configs.d_model * configs.d_channel,
                                    configs.d_output)

        self.pe = configs.pe
        self._d_input = configs.d_input
        self._d_model = configs.d_model

        # MLP Layer - To generate Projector(.); to Obtain series-wise representations
        self.dense = nn.Sequential(
            nn.Linear(240128, 256),  # 240128 = encoder1 out features + encoder2 out features
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

        if self.training_mode == 'pre_train':
            self.awl = AutomaticWeightedLoss(2)
            self.contrastive = ContrastiveWeight(args)
            self.aggregation = AggregationRebuild(args)
            # self.head = nn.Linear(240128, 336)  # Reconstruction, we have 336 time steps
            self.head = nn.Linear(240128, configs.d_input * configs.d_channel)  # Replaced to handle multi-variate
            self.mse = torch.nn.MSELoss()
            
    def forward(self, stage, x_in_t, pre_train=False):

        # x_in_t: (128, 336, 133)
        encoding_1 = self.embedding_channel(x_in_t)  # (128, 336, 512)
        input_to_gather = encoding_1 

        if self.pe:
            pe = torch.ones_like(encoding_1[0])
            position = torch.arange(0, self._d_input).unsqueeze(-1)
            temp = torch.Tensor(range(0, self._d_model, 2))
            temp = temp * -(math.log(10000) / self._d_model)
            temp = torch.exp(temp).unsqueeze(0)
            temp = torch.matmul(position.float(), temp)  # shape:[input, d_model/2]
            pe[:, 0::2] = torch.sin(temp)
            pe[:, 1::2] = torch.cos(temp)

            encoding_1 = encoding_1 + pe  # (128, 336, 512)

        for encoder in self.encoder_list_1:
            # encoding_1: (128, 336, 512)
            encoding_1, score_input = encoder(encoding_1, stage)
            
        encoding_2 = self.embedding_input(x_in_t.transpose(-1, -2))  # encoding_2: (128, 133, 512)
        channel_to_gather = encoding_2  

        for encoder in self.encoder_list_2:
            # encoding_2: (128, 133, 512)
            encoding_2, score_channel = encoder(encoding_2, stage)

        encoding_1 = encoding_1.reshape(encoding_1.shape[0], -1)  # (128, 172032)
        encoding_2 = encoding_2.reshape(encoding_2.shape[0], -1)  # (128, 68096)
        
        encoding_concat = self.gate(torch.cat([encoding_1, encoding_2], dim=-1))  # (128, 2)
        
        # gate: torch.Size([128, 2])
        gate = F.softmax(encoding_concat, dim=-1)  
        encoding = torch.cat([encoding_1 * gate[:, 0:1], encoding_2 * gate[:, 1:2]], dim=-1)  # (128, 240128)
        print(encoding.shape)
        
        # Projections
        projections = self.dense(encoding)  # (128, 128)

        if pre_train:
            # loss_cl: torch.Size([])
            # similarity_matrix: torch.Size([128, 128])
            # logits: torch.Size([128, 127])
            # positives_mask: torch.Size([128, 128])
            loss_cl, similarity_matrix, logits, positives_mask = self.contrastive(projections)           
            
            # rebuild_weight_matrix: torch.Size([128, 128])
            # agg_x: torch.Size([128, 240128])
            rebuild_weight_matrix, agg_x = self.aggregation(similarity_matrix, encoding)
            
            # pred_x: torch.Size([128, 336])
            pred_x = self.head(agg_x.reshape(agg_x.size(0), -1))
            
            # x_in_t.shape: torch.Size([128, 336, 133])
            # x_in_t.reshape(x_in_t.size(0), -1): torch.Size([128, 44688])
            loss_rb = self.mse(pred_x, x_in_t.reshape(x_in_t.size(0), -1).detach())
            loss = self.awl(loss_cl, loss_rb)

            return loss, loss_cl, loss_rb

        return encoding_concat, encoding
    

In [7]:

def model_pretrain(model, model_optimizer, model_scheduler, train_loader, configs, args, device):
    total_loss = []
    total_cl_loss = []
    total_rb_loss = []
    
    model.to(device)
    model.train()
    for batch_idx, (data, labels) in tqdm.tqdm(enumerate(train_loader), desc="Pre-training model", total=len(train_loader)):  # data shape: (batch_size, seqs, channels)

        model_optimizer.zero_grad()
        # When masking, data is reshaped to (batch_size, channel, seqs) - Inside the data_transform_masked4cl()
        data_masked_m, mask = data_transform_masked4cl(data, args.masking_ratio, args.lm, args.positive_nums)
        data_masked_om = torch.cat([data, data_masked_m], 0)  # (batch_size, seqs, channels)

        data, labels = data.float().to('cpu'), labels.float().to('cpu')
        data_masked_om = data_masked_om.float().to(device)

        # Produce embeddings of original and masked samples  (data_masked_om = data samples + masked samples)
        # loss, loss_cl, loss_rb = model(data_masked_om, pretrain=True)
        # return loss, loss_cl, loss_rb
        
        encoding_concat, encodings = model(stage='train', x_in_t=data_masked_om, pre_train=True)
        
        return encoding_concat, encodings

    #     loss.backward()
    #     model_optimizer.step()
    # 
    #     total_loss.append(loss.item())
    #     total_cl_loss.append(loss_cl.item())
    #     total_rb_loss.append(loss_rb.item())
    # 
    # total_loss = torch.tensor(total_loss).mean()
    # total_cl_loss = torch.tensor(total_cl_loss).mean()
    # total_rb_loss = torch.tensor(total_rb_loss).mean()
    # 
    # model_scheduler.step()
    # 
    # return total_loss, total_cl_loss, total_rb_loss


In [8]:
sepsis_data = Load_Dataset(all_patients, TSlength_aligned=336, training_mode='pre_train')
train_loader = torch.utils.data.DataLoader(dataset=sepsis_data, batch_size=32, shuffle=True, 
                                           drop_last=True, num_workers=4)  # (32, 336, 133)


In [9]:
for i, j in train_loader:
    print(i.shape)
    break

torch.Size([32, 336, 40])


# Args

In [10]:
# from models.simmtm.model import TFC
from models.simmtm.config import Config

model = TFC(configs=Config(), args=args)
params_group = [{'params': model.parameters()}]
model_optimizer = torch.optim.Adam(params_group, lr=args.pretrain_lr, betas=(Config().beta1, Config().beta2),
                                       weight_decay=0)
model_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=model_optimizer, T_max=args.pretrain_epoch)
encoding_concat, encodings = model_pretrain(model=model, model_optimizer=model_optimizer, model_scheduler=model_scheduler, 
               train_loader=train_loader, configs=Config(), args=args, device='cuda')


Pre-training model:   0%|          | 0/635 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 442.00 MiB. GPU 

In [ ]:
# loss, loss_cl, loss_rb = model_pretrain(model=model, model_optimizer=model_optimizer, model_scheduler=model_scheduler, 
#                train_loader=train_loader, configs=Config(), args=args, device='cuda')